# 02 - Chao1 Estimator Analysis

This notebook applies the Chao1 species richness estimator to estimate the total number of human rights defenders, including those not captured in historical records.

## Background

The **Chao1 estimator** is a non-parametric method from ecology used to estimate species richness. We apply it here to estimate the "dark figure" of human rights defenders - those who existed but are not recorded in Wikidata.

**Formula:** S_chao1 = S_obs + (f1²) / (2 × f2)

Where:
- S_obs = observed number of individuals
- f1 = singletons (individuals with 1 work)
- f2 = doubletons (individuals with 2 works)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (14, 8)
plt.rcParams['font.size'] = 12

## 1. Load Data

In [ ]:
df = pd.read_csv('../data/humans_rigts_defenders_with_works_count.csv')
print(f"Loaded {len(df)} records")
df.head()

## 2. Chao1 Estimator Function

In [ ]:
def calculate_chao1(works_counts):
    """
    Calculate Chao1 estimator for species richness.
    
    Args:
        works_counts: Array of work counts for each individual
    
    Returns:
        Tuple of (chao1_estimate, observed_richness, percent_captured)
    """
    counts = works_counts.copy()
    counts[counts == 0] = 1  # Treat 0 works as 1 work
    counts = counts[~np.isnan(counts)]
    
    if len(counts) == 0:
        return 0, 0, 0
    
    observed = len(counts)
    f1 = np.sum(counts == 1)  # singletons
    f2 = np.sum(counts == 2)  # doubletons
    
    if f2 > 0:
        chao1_estimate = observed + (f1 ** 2) / (2 * f2)
    else:
        if f1 > 1:
            chao1_estimate = observed + (f1 * (f1 - 1)) / 2
        else:
            chao1_estimate = observed
    
    percent_captured = (observed / chao1_estimate) * 100 if chao1_estimate > 0 else 100
    
    return chao1_estimate, observed, percent_captured

## 3. Analysis by Decade (Worldwide)

In [ ]:
# Prepare data
df_filtered = df[
    (df['productive_year'].notna()) & 
    (df['works_count'].notna()) &
    (df['productive_year'] <= 2020)
].copy()

df_filtered['decade'] = (df_filtered['productive_year'] // 10) * 10

# Calculate Chao1 for each decade
results = []
for decade in sorted(df_filtered['decade'].unique()):
    decade_data = df_filtered[df_filtered['decade'] == decade]
    
    if len(decade_data) < 5:  # Skip small samples
        continue
    
    works = decade_data['works_count'].values
    chao1_est, observed, percent_cap = calculate_chao1(works)
    
    results.append({
        'decade': int(decade),
        'observed_richness': observed,
        'chao1_estimate': chao1_est,
        'percent_captured': percent_cap,
        'individuals_0_works': np.sum(works == 0),
        'individuals_1_work': np.sum(works == 1)
    })

results_df = pd.DataFrame(results)
results_df

In [ ]:
# Visualize: Observed vs Estimated
results_display = results_df[results_df['decade'] <= 2010]

fig, ax = plt.subplots(figsize=(14, 8))

ax.plot(results_display['decade'], results_display['observed_richness'], 
        marker='o', linewidth=2.5, markersize=8, label='Observed', color='#1976D2')
ax.plot(results_display['decade'], results_display['chao1_estimate'], 
        marker='s', linewidth=2.5, markersize=8, label='Chao1 Estimate', 
        color='#D32F2F', linestyle='--')
ax.fill_between(results_display['decade'], 
                results_display['observed_richness'], 
                results_display['chao1_estimate'], 
                alpha=0.2, color='#FFA726', label='Unobserved (estimated)')

# Add percent captured labels
for _, row in results_display.iterrows():
    ax.text(row['decade'], row['chao1_estimate'] + 500, 
            f"{row['percent_captured']:.1f}%", 
            ha='center', va='bottom', fontsize=9, color='#D32F2F')

ax.set_xlabel('Decade', fontsize=14)
ax.set_ylabel('Number of Individuals', fontsize=14)
ax.set_title('Chao1 Estimator: Observed vs Estimated Human Rights Defenders', fontsize=16)
ax.legend(fontsize=12, loc='upper left')
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('../figures/chao1_worldwide.png', dpi=150, bbox_inches='tight')
plt.show()

## 4. Analysis by Country

In [ ]:
# Get top 10 countries
df_with_country = df_filtered[df_filtered['modern_country'].notna()].copy()
top_countries = df_with_country['modern_country'].value_counts().head(10).index.tolist()

# Calculate Chao1 for each country and decade
country_results = []

for country in top_countries:
    country_data = df_with_country[df_with_country['modern_country'] == country]
    
    for decade in sorted(country_data['decade'].unique()):
        decade_data = country_data[country_data['decade'] == decade]
        
        if len(decade_data) < 5:
            continue
        
        works = decade_data['works_count'].values
        chao1_est, observed, percent_cap = calculate_chao1(works)
        
        country_results.append({
            'country': country,
            'decade': int(decade),
            'chao1_estimate': chao1_est,
            'observed_richness': observed
        })

country_df = pd.DataFrame(country_results)
country_df.head(10)

In [ ]:
# Visualize by country
fig, ax = plt.subplots(figsize=(16, 10))

colors = plt.cm.tab10(np.linspace(0, 1, len(top_countries)))

for i, country in enumerate(top_countries):
    data = country_df[country_df['country'] == country].sort_values('decade')
    if len(data) > 0:
        ax.plot(data['decade'], data['chao1_estimate'], 
                marker='o', linewidth=2.5, markersize=8, 
                color=colors[i], alpha=0.8, label=country)

ax.set_xlabel('Decade', fontsize=14)
ax.set_ylabel('Chao1 Estimate', fontsize=14)
ax.set_title('Estimated Human Rights Defenders by Country (Chao1)', fontsize=16)
ax.legend(fontsize=11, loc='upper left')
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('../figures/chao1_by_country.png', dpi=150, bbox_inches='tight')
plt.show()

## 5. Save Results

In [ ]:
results_df.to_csv('../data/chao1_results_world.csv', index=False)
print("Saved results to data/chao1_results_world.csv")

## 6. Key Findings

The Chao1 analysis reveals:

1. **Low capture rates**: Only a small percentage of human rights defenders are captured in historical records
2. **Temporal variation**: Capture rates vary significantly across decades
3. **Geographic disparities**: Some countries have much higher capture rates than others